# Proyecto 1
Flavio Galán 22386
Josue Say
Isabella Miralles

## Descarga de los Datos
Para descargar los datos se utiliza Selenium y Python, además se tiene un sistema de cacheo que funciona en base al archivo `links_cache.txt`. Si este archivo ya existe entonces no se realizará ninguna descarga ni descompresión de los datos.

In [ ]:
import os
import subprocess
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

cache_file_name = "links_cache.txt"
zip_dir = "zips/"
files_dir = "data/"

links = []
if os.path.exists(cache_file_name):
    with open(cache_file_name, "r") as file:
        content = file.read()
        links = eval(content)
else:
    with webdriver.Firefox() as driver:
        driver.implicitly_wait(3)

        driver.get("http://www.mineduc.gob.gt/BUSCAESTABLECIMIENTO_GE/")
        assert "Portal SAT" in driver.title

        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By., "Descargar"))
        )
        elems = driver.find_elements(By.LINK_TEXT, "Descargar")
        for elem in elems:
            link = elem.get_property("href")
            if "2025" in link or "2024" in link:
                print(link)
                links.append(link)

    lines = ["["]
    for link in links:
        line = f'\t"{link}",\n'
        lines.append(line)
    lines.append("]")

    with open(cache_file_name, "w") as file:
        file.writelines(lines)

    for link in links:
        subprocess.run(["mkdir", "-p", zip_dir])
        filename = zip_dir + link.split("/")[-1]
        subprocess.run(["curl", link, "--output", filename])

        subprocess.run(["unzip", "-d", files_dir, filename])